In [3]:
import os
import time
from googleapiclient.discovery import build
import json
import itertools
import networkx as nx
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [4]:
DEVELOPER_KEY = open("../esercizi_classe/api_key.txt").read()
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [5]:
def getIDfromName(name):
    request = youtube.search().list(
        part="snippet",
        q=name,
        type="channel",
        maxResults=5
    )
    response = request.execute()
    return response['items'][0]['id']['channelId']
name = "TheMerluzz"
channel_id = getIDfromName(name)


In [6]:
def getChannelPlaylist(channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
playlist_id = getChannelPlaylist(channel_id)
playlist_id

'UUBzFCho7e5KLWlfoV-V3p7Q'

In [7]:
def getLast100Videos(playlist_id):
    video_ids = []
    nextPageToken = None
    while len(video_ids) < 100:
        if nextPageToken:
            request = youtube.playlistItems().list(
                part="snippet",
                playlistId=playlist_id,
                maxResults=50,
                pageToken=nextPageToken
            )
        else:
            request = youtube.playlistItems().list(
                part="snippet",
                playlistId=playlist_id,
                maxResults=50
            )
        response = request.execute()
        video_ids += [item['snippet']['resourceId']['videoId'] for item in response['items']]
        nextPageToken = response.get('nextPageToken')
    return video_ids
video_ids = getLast100Videos(playlist_id)
print(len(video_ids))

100


In [8]:
def getCommentatorsForVideo(video_id,number):
    commentators, nextPageToken = OnePageCommentators(video_id,number)
    
    while nextPageToken:
        newComments = []
        newComments, nextPageToken = OnePageCommentators(video_id, nextPageToken=nextPageToken, number=number)
        commentators.extend(newComments)
    return commentators

def OnePageCommentators(video_id, number, nextPageToken=None):
    commentators = []
    if nextPageToken:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=number,
            pageToken=nextPageToken
        )
    else:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=number
        )
    
    response = request.execute()
    # print("Response items: ", len(response['items']))
    for item in response['items']:
        commentators.append(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])
        # if 'replies' in item:
        #     # print("Replies", len(item['replies']['comments']))
        #     for reply in item['replies']['comments']:
        #         commentators.append(reply['snippet']['authorDisplayName'])
    
    nextPageToken = response.get('nextPageToken', None)
    
    return commentators, nextPageToken

def getCommentatorsForAllVideos(video_ids,number):
    commentators = dict()
    sum = 0
    for video_id in video_ids:
        try:
            commentators[video_id] = getCommentatorsForVideo(video_id,number)
            print(video_id, len(commentators[video_id]))
            sum += len(commentators[video_id])
        except Exception as e:
            print(e)
            continue
    print(sum)
    return commentators

commenters = getCommentatorsForAllVideos(video_ids,100)


FtR5ItvfwPo 6
ctfbt7Teew4 253
T_W2UDjV7jA 33
ZN0FBpfJSDI 31
hiAeVH32xFg 174
hpA3WWZAbC8 19
Zdna0k812M8 105
EEquT7LzTRQ 42
HsNooSGlfFs 313
BY9ppf2M3kE 17
ptKHfphOnA8 102
QNR5zQstD9k 23
NF9JqlB5fCk 128
EFxzpD1iq6I 35
CVTnit4u53I 53
Ai9AtLzGLqg 18
cxXXEf3G_Gc 11
ekeVWvV-hyo 284
p1u0PyBA_gg 80
DaEuAIrbG5U 152
Xx-OJ_I2DSI 49
ncyO4QyFGXU 101
O2czpHCkARI 21
HJtktoJOY1Q 123
XYZG5VAxtFI 311
h1EQDPpUFxk 239
vvEFWCyGs4Y 263
gfgLy48PwJM 48
bkU_FMrITWY 85
s552weZLwgg 11
8IuTnVroU3M 171
NyFy5hmSKak 154
hA6eZObs-VU 57
GLCBJehRyr4 36
YxJ-bnTTuL0 3
CZnSVTpfLpc 95
Rl4onTr3IAc 136
wxSoIP5amzw 55
4IwEDaG28uA 15
dWW9IvtBOqc 296
RjMTlVMgEUg 9
buMfenWth7I 99
ey6L3O7VWYg 28
WhUwJKCAsxk 53
fEvaSYbWQaU 55
GOXbwHPxgvs 421
ikgG2QvjnjQ 11
LXvrw-HkutI 72
Q-oTpChMjPE 26
FSGzN-wgo6Q 48
V1alugwhkR8 105
152AXSdIQxM 63
phIi_8FeUT0 31
zd-O24m_60o 133
_R8f_GIZFTI 30
FNk7rtqdxMU 213
RUJD3aAMATM 8
h3t0vBgDyLY 197
KG6Xk6sKTGU 61
G1yyfGEmd7w 73
UzP27aY653Y 5
wrgc2BrRVF0 143
6iV6XuWWw7o 12
yhwnFNb9uys 102
7NsB4IXDrFQ 21
_K0OJm

In [ ]:
def get_words_from_comments(video_id):
    commenters_words = dict()
    nextPageToken = None
    
    while True:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=100,
                pageToken=nextPageToken
            )
            response = request.execute()
            
            for item in response['items']:
                author = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
                text = item['snippet']['topLevelComment']['snippet']['textDisplay'].split()
                text = [word.lower() for word in text]
                if author in commenters_words:
                    commenters_words[author].extend(text)
                else:
                    commenters_words[author] = text
            
            nextPageToken = response.get('nextPageToken')
            if not nextPageToken:
                break  # No more pages, exit loop
        
        except Exception as e:
            print(f"Error fetching comments for video {video_id}: {e}")
            break
    
    return commenters_words


def get_words_from_all_videos(video_ids):
    commenters_words = dict()
    
    for video_id in video_ids:
        try:
            words_dict = get_words_from_comments(video_id)
            for author, words in words_dict.items():
                if author in commenters_words:
                    commenters_words[author].update(words)
                else:
                    commenters_words[author] = set(words)
        except Exception as e:
            print(f"Error processing video {video_id}: {e}")
            continue
    
    return commenters_words


commenters_words = get_words_from_all_videos(video_ids)
print(commenters_words)


In [ ]:
def make_first_interaction_graph(commenters_words):
    G = nx.Graph()
    for author1, author2 in itertools.combinations(commenters_words.keys(), 2):
        words1 = commenters_words[author1]
        words2 = commenters_words[author2]
        weight = get_weight(words1, words2)
        if weight > 0:
            G.add_edge(author1, author2, weight=weight)
    return G

def get_weight(words1, words2):
    return abs(len(words1.intersection(words2))/len(words1.union(words2)))

Grafo = make_first_interaction_graph(commenters_words)

nx.write_gexf(Grafo, "grafo.gexf")


In [ ]:
len(Grafo.edges()), len(Grafo.nodes())


In [9]:

def get_videos_and_titles(playlist_id):
    titles = dict()
    nextPageToken = None
    while len(titles) < 100:
        if nextPageToken:
            request = youtube.playlistItems().list(
                part="snippet",
                playlistId=playlist_id,
                maxResults=50,
                pageToken=nextPageToken
            )
        else:
            request = youtube.playlistItems().list(
                part="snippet",
                playlistId=playlist_id,
                maxResults=50
            )
        response = request.execute()
        for item in response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            title = item['snippet']['title']
            titles[video_id] = title
        nextPageToken = response.get('nextPageToken')
    return titles

def get_videos_and_words(titles):
    videos_title_words = dict()
    for video_id, title in titles.items():
        words = title.split()
        words = [word.lower() for word in words]
        videos_title_words[video_id] = words
    return videos_title_words
    

video_titles = get_videos_and_titles(playlist_id)
video_words = get_videos_and_words(video_titles)
for video_id in video_titles:
    print(video_id)

FtR5ItvfwPo
ctfbt7Teew4
T_W2UDjV7jA
ZN0FBpfJSDI
hiAeVH32xFg
hpA3WWZAbC8
Zdna0k812M8
EEquT7LzTRQ
HsNooSGlfFs
BY9ppf2M3kE
ptKHfphOnA8
QNR5zQstD9k
NF9JqlB5fCk
EFxzpD1iq6I
CVTnit4u53I
Ai9AtLzGLqg
cxXXEf3G_Gc
ekeVWvV-hyo
p1u0PyBA_gg
DaEuAIrbG5U
Xx-OJ_I2DSI
ncyO4QyFGXU
O2czpHCkARI
HJtktoJOY1Q
XYZG5VAxtFI
h1EQDPpUFxk
vvEFWCyGs4Y
gfgLy48PwJM
bkU_FMrITWY
s552weZLwgg
8IuTnVroU3M
NyFy5hmSKak
hA6eZObs-VU
GLCBJehRyr4
YxJ-bnTTuL0
CZnSVTpfLpc
Rl4onTr3IAc
wxSoIP5amzw
4IwEDaG28uA
dWW9IvtBOqc
RjMTlVMgEUg
buMfenWth7I
ey6L3O7VWYg
WhUwJKCAsxk
fEvaSYbWQaU
GOXbwHPxgvs
ikgG2QvjnjQ
LXvrw-HkutI
Q-oTpChMjPE
FSGzN-wgo6Q
V1alugwhkR8
152AXSdIQxM
phIi_8FeUT0
zd-O24m_60o
_R8f_GIZFTI
FNk7rtqdxMU
RUJD3aAMATM
h3t0vBgDyLY
KG6Xk6sKTGU
G1yyfGEmd7w
UzP27aY653Y
wrgc2BrRVF0
6iV6XuWWw7o
yhwnFNb9uys
7NsB4IXDrFQ
_K0OJmusDLo
h_SPWSMaGec
Kpw3C3LUwdw
wa3b9LBQkOQ
OjsnqAuKEmw
VBQNtUGPvzE
58VqRBy9rDg
xgSg08kaWPQ
BhQRb2uFbd0
BdyFlMN3fMU
7ySoAZMXXB0
YZ_4NmXWU-I
mZALCV1fmKo
1FS-hO6a-Og
Q2eHclv25Ys
_NkKXQDNLU4
5CrL6xILeLI
rSmysFCItUk
NX44

In [10]:
def get_commenters_set(commenters):
    commenters_set = set()
    for video_id in commenters:
        commenters_set.update(commenters[video_id])
    return commenters_set

In [11]:
def get_commenters_dict(commenters, video_titles):
    commenters_dict = dict()
    for video_id in commenters:
        for commenter in commenters[video_id]:
            if commenter in commenters_dict:
                commenters_dict[commenter].add((video_id, video_titles[video_id]))
            else:
                commenters_dict[commenter] = set([(video_id, video_titles[video_id])])
    return commenters_dict

commenters_dict = get_commenters_dict(commenters, video_titles)


In [12]:
len(commenters_dict)

7648

In [13]:
def shared_videos_graph(commenters_dict,video_words):
    G = nx.Graph()
    for author1, author2 in itertools.combinations(commenters_dict.keys(), 2):
        videos1 = commenters_dict[author1]
        videos2 = commenters_dict[author2]
        shared_videos = videos1.intersection(videos2)
        if shared_videos:
            if len(shared_videos) > 1:
                G.add_edge(author1, author2, weight=len(shared_videos),sim=get_sim(shared_videos, video_words))
    return G

def get_sim(shared_videos, video_words):
    sim = 0
    for video1, video2 in itertools.combinations(shared_videos, 2):
        words1 = video_words[video1[0]]
        words2 = video_words[video2[0]]
        intersection_len = len([word for word in words1 if word in words2])
        union_len = len(words1) + len(words2) - intersection_len
        sim += intersection_len/union_len
    return sim

video_words = get_videos_and_words(video_titles)
Grafo2 = shared_videos_graph(commenters_dict, video_words)



In [14]:
len(Grafo2.edges()), len(Grafo2.nodes())

(9009, 1032)

In [15]:
nx.write_gexf(Grafo2, "grafo2.gexf")

In [ ]:
import json
for edge in Grafo2.edges():
    # print the edge and the sim value
    if Grafo2[edge[0]][edge[1]]['weight'] > 1:
        # add all the edges to json file
        with open('edges.json', 'a') as f:
            f.write(json.dumps({'source': edge[0], 'target': edge[1], 'weight': Grafo2[edge[0]][edge[1]]['weight'], 'sim': Grafo2[edge[0]][edge[1]]['sim']}))
            f.write('\n')